In [3]:
import requests
import pandas as pd
import json

In [4]:
launches_url = "https://api.spacexdata.com/v4/launches"     

In [5]:
response_1 = requests.get(launches_url)
launch_data = response_1.json()

In [6]:
target_key = "cores"
dataframes = []
for launch in launch_data:
    if target_key in launch:
        cores_data = launch[target_key]
        for core in cores_data:
            core['id'] = launch['id'] # Adding the 'id' field from the launch to each core
            core['rocket'] = launch['rocket']
            core['launchpad'] = launch['launchpad']
            core['payloads'] = launch['payloads']
            core['Date'] = pd.to_datetime(launch['date_utc'].split('T')[0])  # Extract year, month and day and convert to time type
        df = pd.json_normalize(cores_data)
        dataframes.append(df)

In [7]:
if dataframes:
    launch_cores = pd.concat(dataframes, ignore_index=True)
    print(launch_cores)

                         core flight gridfins   legs reused landing_attempt  \
0    5e9e289df35918033d3b2623      1    False  False  False           False   
1    5e9e289ef35918416a3b2624      1    False  False  False           False   
2    5e9e289ef3591814873b2625      1    False  False  False           False   
3    5e9e289ef3591855dc3b2626      1    False  False  False           False   
4    5e9e289ef359184f103b2627      1    False  False  False           False   
..                        ...    ...      ...    ...    ...             ...   
210                      None   None     None   None   None            None   
211                      None   None     True   True  False            None   
212                      None   None     True   True  False            None   
213                      None   None     True   True  False            None   
214                      None   None     None   None   None            None   

    landing_success landing_type landpad           

In [8]:
launches_cores  = launch_cores.loc[:, ["Date","core","flight","gridfins","reused","legs","landing_success","landing_type","landpad","id","rocket","launchpad","payloads"]]
column_mapping = {
    'flight': 'Flights',
    'gridfins': 'GridFins',
    'reused': 'Reused',
    'legs': 'Legs',
    'landpad':'LandingPad'
}
# Rename the columns using the rename method
launches_cores.rename(columns=column_mapping, inplace=True)


In [9]:
# Combine 'landing_success' and 'landing_type' into a new column 'Outcome'
launches_cores['Outcome'] = launches_cores['landing_success'].astype(str) + ' - ' + launches_cores['landing_type'].astype(str)

# Move the 'Outcome' column to the first position
columns = ['Outcome'] + [col for col in launches_cores.columns if col != 'Outcome']
launches_cores = launches_cores[columns]
# Drop the 'landing_success' and 'landing_type' columns
launches_cores.drop(['landing_success', 'landing_type'], axis=1, inplace=True)
print(launches_cores)

         Outcome       Date                      core Flights GridFins Reused  \
0    None - None 2006-03-24  5e9e289df35918033d3b2623       1    False  False   
1    None - None 2007-03-21  5e9e289ef35918416a3b2624       1    False  False   
2    None - None 2008-08-03  5e9e289ef3591814873b2625       1    False  False   
3    None - None 2008-09-28  5e9e289ef3591855dc3b2626       1    False  False   
4    None - None 2009-07-13  5e9e289ef359184f103b2627       1    False  False   
..           ...        ...                       ...     ...      ...    ...   
210  None - None 2022-12-01                      None    None     None   None   
211  None - None 2022-12-01                      None    None     True  False   
212  None - None 2022-12-01                      None    None     True  False   
213  None - None 2022-12-01                      None    None     True  False   
214  None - None 2022-12-01                      None    None     None   None   

      Legs LandingPad      

In [10]:
payloads_url = "https://api.spacexdata.com/v4/payloads"
response_2 = requests.get(payloads_url)
payloads_data = response_2.json()
df_payloads = pd.read_json(json.dumps(payloads_data))
payloads = df_payloads.loc[:, ["mass_kg","orbit","id"]]
column_mapping = {
    'mass_kg': 'PayloadMass',
    'orbit': 'Orbit'
}
# Rename the columns using the rename method
payloads.rename(columns=column_mapping, inplace=True)
print(payloads.to_string())

     PayloadMass  Orbit                        id
0          20.00    LEO  5eb0e4b5b6c3bb0006eeb1e1
1            NaN    LEO  5eb0e4b6b6c3bb0006eeb1e2
2            NaN    LEO  5eb0e4b6b6c3bb0006eeb1e3
3            NaN    LEO  5eb0e4b6b6c3bb0006eeb1e4
4         165.00    LEO  5eb0e4b7b6c3bb0006eeb1e5
5         200.00    LEO  5eb0e4b7b6c3bb0006eeb1e6
6            NaN    LEO  5eb0e4b7b6c3bb0006eeb1e7
7            NaN    LEO  5eb0e4b9b6c3bb0006eeb1e8
8            NaN    LEO  5eb0e4b9b6c3bb0006eeb1e9
9         525.00    LEO  5eb0e4bab6c3bb0006eeb1ea
10        400.00    ISS  5eb0e4bab6c3bb0006eeb1eb
11        400.00    LEO  5eb0e4bab6c3bb0006eeb1ec
12        677.00    ISS  5eb0e4bbb6c3bb0006eeb1ed
13        500.00     PO  5eb0e4bbb6c3bb0006eeb1ee
14       3170.00    GTO  5eb0e4bbb6c3bb0006eeb1ef
15       3325.00    GTO  5eb0e4bbb6c3bb0006eeb1f0
16       2296.00    ISS  5eb0e4bbb6c3bb0006eeb1f1
17       1316.00    LEO  5eb0e4bcb6c3bb0006eeb1f2
18       4535.00    GTO  5eb0e4bcb6c3bb0006eeb1f3


In [11]:
rockets_url = "https://api.spacexdata.com/v4/rockets"
response_3 = requests.get(rockets_url)
rockets_data = response_3.json()
df_rockets = pd.read_json(json.dumps(rockets_data))
rockets = df_rockets.loc[:, ["name","id"]]
column_mapping = {
    'name': 'BoosterVersion'
}
# Rename the columns using the rename method
rockets.rename(columns=column_mapping, inplace=True)
rockets

,BoosterVersion,id
0,Falcon 1,5e9d0d95eda69955f709d1eb
1,Falcon 9,5e9d0d95eda69973a809d1ec
2,Falcon Heavy,5e9d0d95eda69974db09d1ed
3,Starship,5e9d0d96eda699382d09d1ee


In [12]:
cores_url = "https://api.spacexdata.com/v4/cores"
response_4 = requests.get(cores_url)
cores_data = response_4.json()
df_cores = pd.read_json(json.dumps(cores_data))
cores = df_cores.loc[:, ["block", "reuse_count","serial","id"]]
column_mapping = {
    'block': 'Block',
    'reuse_count': 'ReusedCount',
    'serial': 'Serial'
}
# Rename the columns using the rename method
cores.rename(columns=column_mapping, inplace=True)
cores

,Block,ReusedCount,Serial,id
0,NaN,0,Merlin1A,5e9e289df35918033d3b2623
1,NaN,0,Merlin2A,5e9e289ef35918416a3b2624
2,NaN,0,Merlin1C,5e9e289ef3591814873b2625
3,NaN,0,Merlin2C,5e9e289ef3591855dc3b2626
4,NaN,0,Merlin3C,5e9e289ef359184f103b2627
...,...,...,...,...
78,5.0,2,B1073,627843db57b51b752c5c5a54
79,5.0,0,B1074,627843de57b51b752c5c5a55
80,5.0,0,B1075,627843e257b51b752c5c5a56
81,5.0,0,B1076,633d9da235a71d1d9c66797a


In [13]:
launchpads_url = "https://api.spacexdata.com/v4/launchpads"
response_5 = requests.get(launchpads_url)
launchpads_data = response_5.json()
df_launchpads= pd.read_json(json.dumps(launchpads_data))
launchpads = df_launchpads.loc[:, ["name","longitude","latitude","id"]]
column_mapping = {
    'name': 'LaunchSite',
    'latitude': 'Latitude',
    'longitude': 'Longitude'
}
# Rename the columns using the rename method
launchpads.rename(columns=column_mapping, inplace=True)
launchpads

,LaunchSite,Longitude,Latitude,id
0,VAFB SLC 3W,-120.593144,34.644090,5e9e4501f5090910d4566f83
1,CCSFS SLC 40,-80.577366,28.561857,5e9e4501f509094ba4566f84
2,STLS,-97.156085,25.997264,5e9e4502f5090927f8566f85
3,Kwajalein Atoll,167.743129,9.047721,5e9e4502f5090995de566f86
4,VAFB SLC 4E,-120.610829,34.632093,5e9e4502f509092b78566f87
5,KSC LC 39A,-80.603956,28.608058,5e9e4502f509094188566f88


In [14]:
# Merge two DataFrames based on id and rocket columns
merged_df = pd.merge(launches_cores, rockets, left_on="rocket", right_on="id", how="inner")
# Remove redundant id columns
merged_df.drop(columns=["id_y"], inplace=True)
merged_df.rename(columns={"id_x": "id"}, inplace=True)
merged_df

,Outcome,Date,core,Flights,GridFins,Reused,Legs,LandingPad,id,rocket,launchpad,payloads,BoosterVersion
0,None - None,2006-03-24,5e9e289df35918033d3b2623,1,False,False,False,None,5eb87cd9ffd86e000604b32a,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b5b6c3bb0006eeb1e1],Falcon 1
1,None - None,2007-03-21,5e9e289ef35918416a3b2624,1,False,False,False,None,5eb87cdaffd86e000604b32b,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b6b6c3bb0006eeb1e2],Falcon 1
2,None - None,2008-08-03,5e9e289ef3591814873b2625,1,False,False,False,None,5eb87cdbffd86e000604b32c,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",Falcon 1
3,None - None,2008-09-28,5e9e289ef3591855dc3b2626,1,False,False,False,None,5eb87cdbffd86e000604b32d,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e5],Falcon 1
4,None - None,2009-07-13,5e9e289ef359184f103b2627,1,False,False,False,None,5eb87cdcffd86e000604b32e,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e6],Falcon 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,None - None,2022-11-01,5fe3b8fbb3467846b3242182,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy
211,None - None,2022-11-01,5fe3b906b3467846b3242183,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy
212,None - None,2022-12-01,None,None,True,False,True,None,633f72130531f07b4fdf59c3,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[],Falcon Heavy
213,None - None,2022-12-01,None,None,True,False,True,None,633f72130531f07b4fdf59c3,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[],Falcon Heavy


In [15]:
merged_df_1 = pd.merge(merged_df, launchpads, left_on="launchpad", right_on="id", how="inner")
merged_df_1.drop(columns=["id_y"], inplace=True)
merged_df_1.rename(columns={"id_x": "id"}, inplace=True)
merged_df_1

,Outcome,Date,core,Flights,GridFins,Reused,Legs,LandingPad,id,rocket,launchpad,payloads,BoosterVersion,LaunchSite,Longitude,Latitude
0,None - None,2006-03-24,5e9e289df35918033d3b2623,1,False,False,False,None,5eb87cd9ffd86e000604b32a,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b5b6c3bb0006eeb1e1],Falcon 1,Kwajalein Atoll,167.743129,9.047721
1,None - None,2007-03-21,5e9e289ef35918416a3b2624,1,False,False,False,None,5eb87cdaffd86e000604b32b,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b6b6c3bb0006eeb1e2],Falcon 1,Kwajalein Atoll,167.743129,9.047721
2,None - None,2008-08-03,5e9e289ef3591814873b2625,1,False,False,False,None,5eb87cdbffd86e000604b32c,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",Falcon 1,Kwajalein Atoll,167.743129,9.047721
3,None - None,2008-09-28,5e9e289ef3591855dc3b2626,1,False,False,False,None,5eb87cdbffd86e000604b32d,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e5],Falcon 1,Kwajalein Atoll,167.743129,9.047721
4,None - None,2009-07-13,5e9e289ef359184f103b2627,1,False,False,False,None,5eb87cdcffd86e000604b32e,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e6],Falcon 1,Kwajalein Atoll,167.743129,9.047721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,None - None,2022-11-01,5fe3b8fbb3467846b3242182,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy,KSC LC 39A,-80.603956,28.608058
211,None - None,2022-11-01,5fe3b906b3467846b3242183,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy,KSC LC 39A,-80.603956,28.608058
212,None - None,2022-12-01,None,None,True,False,True,None,633f72130531f07b4fdf59c3,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[],Falcon Heavy,KSC LC 39A,-80.603956,28.608058
213,None - None,2022-12-01,None,None,True,False,True,None,633f72130531f07b4fdf59c3,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[],Falcon Heavy,KSC LC 39A,-80.603956,28.608058


In [16]:
merged_df_2 = pd.merge(merged_df_1, cores, left_on="core", right_on="id", how="inner")
merged_df_2.drop(columns=["id_y"], inplace=True)
merged_df_2.rename(columns={"id_x": "id"}, inplace=True)
merged_df_2

,Outcome,Date,core,Flights,GridFins,Reused,Legs,LandingPad,id,rocket,launchpad,payloads,BoosterVersion,LaunchSite,Longitude,Latitude,Block,ReusedCount,Serial
0,None - None,2006-03-24,5e9e289df35918033d3b2623,1,False,False,False,None,5eb87cd9ffd86e000604b32a,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b5b6c3bb0006eeb1e1],Falcon 1,Kwajalein Atoll,167.743129,9.047721,NaN,0,Merlin1A
1,None - None,2007-03-21,5e9e289ef35918416a3b2624,1,False,False,False,None,5eb87cdaffd86e000604b32b,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b6b6c3bb0006eeb1e2],Falcon 1,Kwajalein Atoll,167.743129,9.047721,NaN,0,Merlin2A
2,None - None,2008-08-03,5e9e289ef3591814873b2625,1,False,False,False,None,5eb87cdbffd86e000604b32c,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,"[5eb0e4b6b6c3bb0006eeb1e3, 5eb0e4b6b6c3bb0006e...",Falcon 1,Kwajalein Atoll,167.743129,9.047721,NaN,0,Merlin1C
3,None - None,2008-09-28,5e9e289ef3591855dc3b2626,1,False,False,False,None,5eb87cdbffd86e000604b32d,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e5],Falcon 1,Kwajalein Atoll,167.743129,9.047721,NaN,0,Merlin2C
4,None - None,2009-07-13,5e9e289ef359184f103b2627,1,False,False,False,None,5eb87cdcffd86e000604b32e,5e9d0d95eda69955f709d1eb,5e9e4502f5090995de566f86,[5eb0e4b7b6c3bb0006eeb1e6],Falcon 1,Kwajalein Atoll,167.743129,9.047721,NaN,0,Merlin3C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,True - RTLS,2019-06-25,5e9e28a6f359188fd53b265e,2,True,True,True,5e9e3032383ecb90a834e7c8,5eb87d35ffd86e000604b37a,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,"[5eb0e4ccb6c3bb0006eeb23a, 5eb0e4ccb6c3bb0006e...",Falcon Heavy,KSC LC 39A,-80.603956,28.608058,5.0,1,B1053
192,False - ASDS,2019-06-25,5e9e28a7f3591878063b2661,1,True,False,True,5e9e3032383ecb6bb234e7ca,5eb87d35ffd86e000604b37a,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,"[5eb0e4ccb6c3bb0006eeb23a, 5eb0e4ccb6c3bb0006e...",Falcon Heavy,KSC LC 39A,-80.603956,28.608058,5.0,0,B1057
193,None - None,2022-11-01,5fe3b8f2b3467846b3242181,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy,KSC LC 39A,-80.603956,28.608058,5.0,0,B1064
194,None - None,2022-11-01,5fe3b8fbb3467846b3242182,1,True,False,True,None,6243aec2af52800c6e91925d,5e9d0d95eda69974db09d1ed,5e9e4502f509094188566f88,[5fe3b86eb3467846b324217c],Falcon Heavy,KSC LC 39A,-80.603956,28.608058,5.0,0,B1065


In [17]:
import numpy as np

merged_df_2['PayloadMass'] = 0
merged_df_2['Orbit'] = np.nan

orbit_dict = {'VLEO':1, 'ISS':2, 'SSO':3, 'PO':4, 'LEO': 5, 'MEO':6, 'GTO':7}
# print(payloads['Orbit'].unique())

s = set()

def check_orbit(pre, aft):
    if not isinstance(pre, str):
        if np.isnan(pre):
            return aft
    if not isinstance(aft, str):
        if np.isnan(aft):
            return pre
    
    if orbit_dict[pre] < orbit_dict[aft]:
        return aft
    else:
        return pre

for index, row in merged_df_2.iterrows():
    pls = row['payloads']
    if len(pls) == 0:
        merged_df_2.at[index, 'PayloadMass'] = np.nan
#         print('no payload found')
        continue
        
    elif len(pls) == 1:
        cur_pl = payloads[payloads['id']==pls[0]].iloc[0]
        merged_df_2.at[index, 'Orbit'] = cur_pl['Orbit']
        merged_df_2.at[index, 'PayloadMass'] = cur_pl['PayloadMass']
        continue
        
    total = 0
    for pl in pls:
        cur_pl = payloads[payloads['id']==pl].iloc[0]
        orbit = cur_pl['Orbit']
        s.add(orbit)
        #update orbit
        merged_df_2.at[index, 'Orbit'] = check_orbit(merged_df_2.at[index, 'Orbit'], orbit)
        
        #update payload
        if not np.isnan(cur_pl['PayloadMass']):
            total += cur_pl['PayloadMass']
    
    if total < 0.001:
        merged_df_2.at[index, 'PayloadMass'] = np.nan
    else:
        merged_df_2.at[index, 'PayloadMass'] = total



In [18]:
new_order=['Date','BoosterVersion','PayloadMass','Orbit','LaunchSite','Outcome','Flights','GridFins','Reused','Legs','LandingPad','Block','ReusedCount','Serial','Longitude','Latitude','id']

In [19]:
chang_data = merged_df_2[new_order]
launch_sorted = chang_data.sort_values(by='Date', ascending=True)
launch_sorted.drop(launch_sorted[launch_sorted['BoosterVersion'] == 'Falcon Heavy'].index, inplace=True)
launch_sorted.dropna(subset=['Date'], inplace=True)

In [20]:
print(launch_sorted)


          Date BoosterVersion  PayloadMass Orbit       LaunchSite  \
0   2006-03-24       Falcon 1         20.0   LEO  Kwajalein Atoll   
1   2007-03-21       Falcon 1          NaN   LEO  Kwajalein Atoll   
2   2008-08-03       Falcon 1          NaN   LEO  Kwajalein Atoll   
3   2008-09-28       Falcon 1        165.0   LEO  Kwajalein Atoll   
4   2009-07-13       Falcon 1        200.0   LEO  Kwajalein Atoll   
..         ...            ...          ...   ...              ...   
157 2022-09-05       Falcon 9      13440.0  VLEO     CCSFS SLC 40   
121 2022-09-11       Falcon 9      14760.0  VLEO       KSC LC 39A   
149 2022-09-17       Falcon 9      13260.0  VLEO     CCSFS SLC 40   
164 2022-09-24       Falcon 9      13260.0  VLEO     CCSFS SLC 40   
187 2022-10-05       Falcon 9          NaN   ISS       KSC LC 39A   

         Outcome Flights GridFins Reused   Legs                LandingPad  \
0    None - None       1    False  False  False                      None   
1    None - None 

In [21]:
launch_sorted.isnull().sum()

Date               0
BoosterVersion     0
PayloadMass       25
Orbit              1
LaunchSite         0
Outcome            0
Flights            0
GridFins           0
Reused             0
Legs               0
LandingPad        36
Block              5
ReusedCount        0
Serial             0
Longitude          0
Latitude           0
id                 0
dtype: int64

In [22]:
launch_sorted.dtypes

Date              datetime64[ns]
BoosterVersion            object
PayloadMass              float64
Orbit                     object
LaunchSite                object
Outcome                   object
Flights                   object
GridFins                  object
Reused                    object
Legs                      object
LandingPad                object
Block                    float64
ReusedCount                int64
Serial                    object
Longitude                float64
Latitude                 float64
id                        object
dtype: object

In [45]:
launch_sorted.to_csv("launch_sorted.csv", index=False)